# Problem 2

In [99]:
import tensorflow as tf

# Explicitly create a Graph object
graph = tf.Graph()

with graph.as_default():
    
    # Variable to keep track of how many times the graph has been run
    global_step = tf.Variable(0, dtype=tf.int32, name="global_step")
        
    # Increments the above `global_step` Variable, should be run whenever the graph is run
    increment_step = global_step.assign_add(1)
        
    # Input layer
    with tf.name_scope("input"):
        # Create input placeholder- takes in a Vector 
        a = tf.placeholder(tf.float32, shape=[None], name="input_placeholder_a")
        b = tf.placeholder(tf.float32, shape=[None], name="input_placeholder_b")
    
    # Operations
    with tf.name_scope("operations"):
        
        # Standard Deviation Operations and Norm
        #for SD tensorflow returns variance, and SD is the square root of variance
        #a norm of a vector is its euclidian lenght
        with tf.name_scope("std_deviation_norm"):
                
            with tf.name_scope("std_deviation_norm_a"):
                _,var_a = tf.nn.moments(a, axes=[0], name="var_a")
                std_dev_a = tf.sqrt(var_a, name="std_dev_a")
                norm_a = tf.norm(a, name = "norm_a")
                
            with tf.name_scope("std_deviation_norm_b"):
                _,var_b = tf.nn.moments(b, axes=[0], name="var_b")
                std_dev_b = tf.sqrt(var_b, name="std_dev_b")
                norm_b = tf.norm(b, name = "norm_b")
        
        # Normalization to unit lenght
        # a vector is normalized to unit lenght dividing it by its norm, by doing that we have
        # a vector with exact same direction, but with lenght 1
        with tf.name_scope("normalize_to_unit_length"):
            
            with tf.name_scope("normalize_to_unit_length_a"):
                norm_unit_a = tf.div(a, norm_a, "norm_unit_a")
                 
            
            with tf.name_scope("normalize_to_unit_length_b"):
                  norm_unit_b = tf.div(b, norm_b, "norm_unit_b") 
            
            #cosine similarity of 2 vectors of unit lenght is 
            #the dot product a x b reduced to a scalar by adding result vector's dimensions
            num_cosine = tf.multiply(norm_unit_a, norm_unit_b)
            cosine_a_b = tf.reduce_sum(tf.multiply(norm_unit_a, norm_unit_b), name = "cosine_a_b")
            
            #covariance is sum((x-mean x) x (y - mean y))/n
            covariance_a_b = tf.reduce_sum(tf.div((tf.multiply (tf.subtract (a, tf.reduce_mean(a)), tf.subtract (b, tf.reduce_mean(b)))),
                                    tf.cast(tf.shape(a), dtype = tf.float32 )))
            
           
            
            
                
    with tf.name_scope("summaries"):
        tf.summary.scalar(tensor = std_dev_a, name="std_dev_a_summary") 
        tf.summary.scalar(tensor = std_dev_b, name="std_dev_b_summary") 
        tf.summary.scalar(tensor = cosine_a_b, name="cosine_a_b_summary") 
        tf.summary.scalar(tensor = covariance_a_b, name="covariance_a_b_summary") 
       
    
    # Global Variables and Operations
    with tf.name_scope("global_ops"):
        # Initialization Op
        init = tf.global_variables_initializer()
        # Collect all summary Ops in graph
        merged_summaries = tf.summary.merge_all()


# Start a Session, using the explicitly created Graph
sess = tf.Session(graph=graph)

# Open a SummaryWriter to save summaries
writer = tf.summary.FileWriter('./problem2_graph', graph)

# Initialize Variables
sess.run(init)





def run_graph(input_tensor_a, input_tensor_b):
    """
    Helper function; runs the graph with given input tensor and saves summaries
    """
    feed_dict = {a: input_tensor_a, b: input_tensor_b}
    summary, step, cos, cov = sess.run([merged_summaries, increment_step, cosine_a_b, covariance_a_b], feed_dict=feed_dict)
    writer.add_summary(summary, global_step=step)
    
    print "Cosine similarity:", cos
    print "Covariance:", cov


# Run the graph with various inputs
#cosine zero
run_graph([32,12],[-6,16])

#cosine 1
run_graph([32,12,33],[16, 6, 16.5])

#random vectors
run_graph([7, 14, 3, 2, -7],[16, 6, 16.5, 14, -18])
run_graph([1, -3, 12, 4, -77,-1000],[1, -16, 9, 1444, -1844, 5])



# Writes the summaries to disk
writer.flush()

# Flushes the summaries to disk and closes the SummaryWriter
writer.close()

# Close the session
sess.close()         
                

Cosine similarity: 0.0
Covariance: -110.0
Cosine similarity: 1.0
Covariance: 46.7778
Cosine similarity: 0.692441
Covariance: 53.68
Cosine similarity: 0.0608351
Covariance: 11979.5
